In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
sns.set(font_scale=2.5)
# 기본 scheme 대신 seaborn scheme 세팅
# 일일이 그래프의 font size를 지정할 필요 없이 seaborn의 font_scale 사용하여 편함

import missingno as msno

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

#### 앞으로 우리가 해야할 프로세스는 대략 아래와 같습니다.
1. **데이터셋 확인** - 대부분의 캐글 데이터들은 잘 정제되어 있습니다. 하지만 가끔 null data가 존재합니다. 이를 확인하고, 향후 수정합니다.
2. **탐색적 데이터 분석(exploratory data analysis)** - 여러 feature 들을 개별적으로 분석하고, feature 들 간의 상관관계를 확인합니다. 여러 시각화 툴을 사용하여 insight를 얻습니다.
3. **feature engineering** - 모델을 세우기에 앞서, 모델의 성능을 높일 수 있도록 feature 들을 engineering 합니다. one-hot encoding, class로 나누기, 구간으로 나누기, 텍스트 데이터 처리 등을 합니다.
4. **model 만들기** - sklearn 을 사용해 모델을 만듭니다. 파이썬에서 머신러닝을 할 때는 sklearn 을 사용하면 수많은 알고리즘을 일관된 문법으로 사용할 수 있습니다. 물론 딥러닝을 위해 tensorflow, pytorch 등을 사용할 수 도 있습니다.
5. **모델 학습 및 예측** - trainset 을 가지고 모델을 학습시킨 후, testset 을 가지고 prediction 합니다.
6. **모델 평가** - 예측 성능이 원하는 수준인지 판단합니다. 풀려는 문제에 따라 모델을 평가하는 방식도 달라집니다. 학습된 모델이 어떤 것을 학습하였는 지 확인해봅니다.

# 1. Dataset 확인

#### pandas
- 파이썬에서 테이블화 된 데이터를 다루는 데 가장 최적화되어 있으며, 많이 쓰이는 라이브러리
- 사용하여 데이터셋의 간단한 통계적 분석 부터, 복잡한 처리들을 간단한 메소드를 사용하여 해낼 수 있다.
- 파이썬으로 데이터분석을 한다고 하면 반드시 능숙해져야 할 라이브러리 -> 반복해서 연습!!<br/><br/>

- 캐글에서 데이터셋은 보통 train set, test set 으로 나뉘어 있습니다.
- 비어져있는 null data가 있는지 확인해줘야 한다.

In [2]:
#Loading csv files
df_train = pd.read_csv('../input/titanic/train.csv')
df_test = pd.read_csv('../input/titanic/test.csv')

In [3]:
df_train.head() # 데이터 셋의 처음 부분을 출력해줌, ()안의 개수만큼 (default = 5)

- PassengerId : 1, 2, 3, ...
- Survived : 생존여부 / target label (0:dead, 1:alive) / int
- Pclass : 티켓의 클래스 / categorical feature (First class, second class, third class), 순서 유 / int
- Name : 이름
- Sex : 성별 / binary (male, female) / str
- Age : 나이 / continuous / int
- SibSp : 함께 탑승한 형제와 배우자의 수 / quantitative (몇 개, 몇 명 같은) / int
- Parch : 함께 탑승한 부모, 아이의 수 / quantitative / int
- Ticket : 티켓 번호 / 하나의 string 데이터 (alphabet + number) / str
- Fare : 탑승료 / continuous / float
- Cabin : 객실 번호 / 카테고리 분류 (alphabet + number) / str
- Embarked : 탑승 항구 / 카테고리(S,C,Q), 순서 무 / str

#### 데이터 타입마다 데이터를 처리하는 방식이 조금씩 달라진다.
##### 이 문제에서
- feature : **Pclass, Age, SibSp, Parch, Fare**
- 예측하려는 target label : **Survived**

In [4]:
df_train.describe() # 각 featurn/column의 간단한 통계적 수치를 알려준다. (ex. min, mean, etc.)

In [5]:
df_test.describe()

테이블을 보면, count가 다 동일한 값이 아니다.  
따라서 숫자가 PassenserID 숫자와 다른 경우, null 값이 존재한다고 볼 수 있다. (null data가 존재하는 열(feature) 존재)  
이를 좀 더 보기 편하도록 그래프로 시각화해서 살펴보자. 

## 1.1 Null data check

### 방법 1 - 숫자 퍼센티지로 확인

In [6]:
for col in df_train.columns:
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100*(df_train[col].isnull().sum() / df_train[col].shape[0]))
    print(msg)
    
    # 각 컬럼마다 null 값의 퍼센트 알아보기 위해
    
    # {:>10} -> format문, 오른쪽으로 정렬
    # .isnull() -> null이냐 (null이면 True, 아니면 False)
    # .sum() -> 앞의 결과에서 (False:0, True:1)이므로 합은 null인 값의 개수를 알려줌]
    # df_train[col].shape[0] -> 전체로 나눠줘야하기 때문에 추가됨

In [7]:
for col in df_test.columns:
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100*(df_test[col].isnull().sum() / df_test[col].shape[0]))
    print(msg)

Train, Test set 에서 Age(둘다 약 20%), Cabin(둘다 약 80%), Embarked(Train만 0.22%) null data 존재  
null data를 채울 수도, 안 채울 수도 있는데 성능에 따라 상황에 따라 다르게 한다. (이는 나중에)  
MSNO 라는 라이브러리를 사용하면 null data의 존재를 더 쉽게 볼 수 있다.

### 방법 2 - msno 사용

In [8]:
msno.matrix(df = df_train.iloc[:,:], figsize = (8,8), color=(0.8, 0.5, 0.2))

# msno는 아래와 같은 matrix 형태를 만들어준다.
# df_train.iloc[:,:] -> 모든 행, 열 선택
# figsize = (8,8) -> 이건 그냥 matrix의 크기

빈 칸이 null이다.

In [9]:
# 바 그래프로도 표현할 수 있다.
msno.bar(df=df_train.iloc[:,:], figsize=(8,8), color = (0.8, 0.5, 0.2))

#### 위 두 가지 MSNO 방법 비교
- matrix - 어느 위치에 null 있는지, 분포 확인 가능  
- bar - 퍼센티지, 개수 등 숫자와 함께 확인 가능

In [10]:
msno.bar(df=df_test.iloc[:,:], figsize=(8,8), color=(0.8,0.5,0.2))

## 1.2 Target label 확인
Target label이 어떤 distribution을 가지고 있는지 확인해봐야 한다.   
Target label이 얼마나 밸런스 있게 data set이 있느냐가 몹시 중요하다.  
Binary classification 문제의 경우, 1과 0의 분포가 어떠냐에 따라 모델의 평가 방법이 달라질 수 있다.

In [11]:
f, ax = plt.subplots(1, 2, figsize = (18,8))
# 그래프를 그릴 때는 항상, 팔레트를 먼저 준비하고 그 위에 그린다.
# subplots -> 큰 틀이 있으면 그 부분부분을 의미, 여기서 1->row, 2->column이므로 1개의 행에 2개의 파트로 이루어진 그래프를 준비하겠다
# figsize = (18,8) -> 가로로 18, 세로로 8

df_train['Survived'].value_counts().plot.pie(explode=[0,0.1], autopct = '%1.1f%%', ax = ax[0], shadow = True)
ax[0].set_title('Pie plot - Survived')
ax[0].set_ylabel('')
sns.countplot('Survived', data = df_train, ax = ax[1])
ax[1].set_title('Count plot - Survived')

plt.show()

# df_train['Survived'].value_counts() -> Survived의 value를 count 하는 것 (여기서는 0 549 | 1 342 즉 series로 각 값이 얼마나 있는지 보여줌)

안타깝게도 죽은 사람이 많다. 38.4 % 가 살아남았다.  

Target label 의 분포가 제법 균일(balanced)합니다. (값을 비교했을 때 극단적이지 않으므로 balance하다.)  
불균일한 경우, 예를 들어서 100중 1이 99, 0이 1개인 경우에는 만약 모델이 모든것을 1이라 해도 정확도가 99%가 나오게 된다.  
0을 찾는 문제라면 이 모델은 원하는 결과를 줄 수 없게 된다. 지금 문제에서는 그렇지 않으니 계속 진행하였다.

# 2. Exploratory data analysis

이제 본격적인 데이터 분석
데이터가 매우 많아 이 많은 데이터 안에 숨겨진 사실을 찾기 위해선 적절한 시각화가 필요하다.  
시각화 라이브러리는 matplotlib, seaborn, plotly 등이 있고, 특정 목적에 맞는 소스코드를 정리해두어 필요할 때마다 참고하면 편하다.  

## 2.1 Pclass
- ordinal, 서수형 데이터
- 카테고리
- 순서가 있는 데이터 타입

먼저 Pclass 에 따른 생존률의 차이를 살펴보았다.  
엑셀의 피벗 차트와 유사한 작업을 하게 되는데, pandas dataframe 에서는 groupby 를 사용하면 쉽게 할 수 있다. 또한 pivot 이라는 메소드도 있다.  
'Pclass', 'Survived' 를 가져온 후, pclass 로 묶는다. 그러고 나면 각 pclass 마다 0, 1 이 count가 되는데, 이를 평균내면 각 pclass 별 생존률이 나온다.  
아래와 같이 count() 를 하면, 각 class 에 몇명이 있는 지 확인할 수 있으며, sum() 을 하면, 216 명중 생존한(survived=1)사람의 총합을 주게 된다.  

In [12]:
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index = True).count()

In [13]:
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index = True).sum()

pandas 의 crosstab 을 사용하면 좀 더 위 과정을 좀 더 수월하게 볼 수 있다.

In [14]:
pd.crosstab(df_train['Pclass'], df_train['Survived'], margins = True).style.background_gradient(cmap='summer_r')

grouped 객체에 mean() 을 하게 되면, 각 클래스별 생존률을 얻을 수 있다. class 1 이면 아래와 같다.  

In [15]:
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index = True).mean().sort_values(by = 'Survived', ascending = False).plot.bar()

보다시피, Pclass 가 좋을 수록(1st) 생존률이 높은 것을 확인할 수 있다.  
좀 더 보기 쉽게 그래프를 그려보겠습니다. seaborn 의 countplot 을 이용하면, 특정 label 에 따른 개수를 확인해볼 수 있습니다.

In [16]:
y_position = 1.02
f, ax = plt.subplots(1,2,figsize=(18,8))
df_train['Pclass'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'], ax = ax[ㅌ0])
ax[0].set_title('Number of Passengers By Pclass', y = y_position)
ax[0].set_ylabel('Count')
sns.countplot('Pclass', hue='Survived', data=df_train, ax = ax[1])
ax[1].set_title('Pclass: Survived vs Dead', y = y_position)
plt.show()

클래스가 높을 수록, 생존 확률이 높은걸 확인할 수 있습니다. Pclass 1, 2, 3 순서대로 63%, 48%, 25% 입니다.  
우리는 생존에 Pclass 가 큰 영향을 미친다고 생각해볼 수 있으며, 나중에 모델을 세울 때 이 feature 를 사용하는 것이 좋을 것이라 판단할 수 있습니다.

## 2.2 Sex
이번에는 성별로 생존률이 어떻게 달라지는 지 확인해보겠습니다.  
마찬가지로 pandas groupby 와 seaborn countplot 을 사용해서 시각화해봅시다.

In [17]:
f, ax = plt.subplots(1,2,figsize=(18,8))
df_train[['Sex', 'Survived']].groupby(['Sex'], as_index = True).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survived vs Sex')
sns.countplot('Sex', hue = 'Survived', data = df_train, ax = ax[1])
ax[1].set_title('Sex: Survived vs Dead')
plt.show()

보시다시피, 여자가 생존할 확률이 높습니다.

In [18]:
df_train[['Sex', 'Survived']].groupby(['Sex'], as_index = False).mean().sort_values(by = 'Survived', ascending = False)

In [19]:
pd.crosstab(df_train['Sex'], df_train['Survived'], margins = True).style.background_gradient(cmap='summer_r')

Pclass 와 마찬가지로, Sex 도 예측 모델에 쓰일 중요한 feature 임을 알 수 있습니다

## 2.3 Both Sex and Pclass
이번에는 Sex, Pclass 두가지에 관하여 생존이 어떻게 달라지는 지 확인해 봅시다.  
seaborn 의 factorplot 을 이용하면, 손쉽게 3개의 차원으로 이루어진 그래프를 그릴 수 있습니다. 

In [20]:
sns.factorplot('Pclass', 'Survived', hue = 'Sex', data = df_train, size = 6, aspect = 1.5)

모든 클래스에서 female 이 살 확률이 male 보다 높은 걸 알 수 있습니다.  
또한 남자, 여자 상관없이 클래스가 높을 수록 살 확률 높습니다.  
위 그래프는 hue 대신 column 으로 하면 아래와 같아집니다.

In [21]:
sns.factorplot(x = 'Sex', y = 'Survived', col = 'Pclass',
              data = df_train, saturation = .5, size = 9, aspect = 1)

## 2.4 Age
이번에는 Age feature 를 살펴봅시다.

In [22]:
print('제일 나이 많은 탑승객 : {:.1f} Years'.format(df_train['Age'].max()))
print('제일 어린 탑승객 : {:.1f} Years'.format(df_train['Age'].min()))
print('탑승객 평균 나이 : {:.1f} Years'.format(df_train['Age'].mean()))

생존에 따른 Age의 histogram 을 그려보겠습니다.

In [23]:
fig, ax = plt.subplots(1,1,figsize = (9, 5))
sns.kdeplot(df_train[df_train['Survived'] == 1]['Age'], ax = ax)
sns.kdeplot(df_train[df_train['Survived'] == 0]['Age'], ax = ax)
plt.legend(['Survived == 1', 'Survived == 0'])
plt.show()

보시다시피, 생존자 중 나이가 어린 경우가 많음을 볼 수 있습니다.

In [24]:
# Age distribution within classes
plt.figure(figsize = (8,6))
df_train['Age'][df_train['Pclass'] == 1].plot(kind = 'kde')
df_train['Age'][df_train['Pclass'] == 2].plot(kind = 'kde')
df_train['Age'][df_train['Pclass'] == 3].plot(kind = 'kde')

plt.xlabel('Age')
plt.title('Age Distribution within classes')
plt.legend(['1st Class', '2nd Class', '3rd Class'])

Class 가 높을 수록 나이 많은 사람의 비중이 커진다.  
나이대가 변하면서 생존률이 어떻게 되는 지 보려고 합니다.  
나이범위를 점점 넓혀가며, 생존률이 어떻게 되는지 한번 봅시다.

In [25]:
cummulate_survival_ratio = []
for i in range(1,80):
    cummulate_survival_ratio.append(df_train[df_train['Age'] < i]['Survived'].sum() / len(df_train[df_train['Age'] < i]['Survived']))

plt.figure(figsize = (7,7))
plt.plot(cummulate_survival_ratio)
plt.title('Survival rate change depending on range of Age', y=1.02)
plt.ylabel('Survival rate')
plt.xlabel('Range of Age(0~x)')
plt.show()

보시다시피, 나이가 어릴 수록 생존률이 확실히 높은것을 확인할 수 있습니다.  
우리는 이 나이가 중요한 feature 로 쓰일 수 있음을 확인했습니다.

## 2.5 Pclass, Sex, Age
지금까지 본, Sex, Pclass, Age, Survived 모두에 대해서 보고싶습니다. 이를 쉽게 그려주는 것이 seaborn 의 violinplot 입니다.  
x 축은 우리가 나눠서 보고싶어하는 case(여기선 Pclass, Sex) 를 나타내고, y 축은 보고 싶어하는 distribution(Age) 입니다.  

In [26]:
f, ax = plt.subplots(1,2,figsize = (18,8))
sns.violinplot("Pclass", "Age", hue = "Survived", data = df_train, scale = 'count', split = True, ax = ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0, 110, 10))
sns.violinplot("Sex", "Age", hue = "Survived", data = df_train, scale = 'count', split = True, ax = ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0, 110, 10))
plt.show()

왼쪽 그림은 Pclass 별로 Age의 distribution 이 어떻게 다른지, 거기에 생존여부에 따라 구분한 그래프입니다.  
오른쪽 그림도 마찬가지 Sex, 생존에 따른 distribution 이 어떻게 다른지 보여주는 그래프입니다.  
생존만 봤을 때, 모든 클래스에서 나이가 어릴 수록 생존을 많이 한것을 볼 수 있습니다.  
오른쪽 그림에서 보면, 명확히 여자가 생존을 많이 한것을 볼 수 있습니다.  
여성과 아이를 먼저 챙긴 것을 볼 수 있습니다.

## 2.6 Embarked
Embarked 는 탑승한 항구를 나타냅니다.  
위에서 해왔던 것과 비슷하게 탑승한 곳에 따르 생존률을 보겠습니다.

In [27]:
f, ax = plt.subplots(1,1,figsize=(7,7))
df_train[['Embarked', 'Survived']].groupby(['Embarked'], as_index = True).mean().sort_values(by='Survived', ascending = False).plot.bar(ax=ax)

보시다시피, 조금의 차이는 있지만 생존률은 좀 비슷한 거 같습니다. 그래도 C가 제일 높군요.  
모델에 얼마나 큰 영향을 미칠지는 모르겠지만, 그래도 사용하겠습니다.  
사실, 모델을 만들고 나면 우리가 사용한 feature 들이 얼마나 중요한 역할을 했는지 확인해볼 수 있습니다. 이는 추후에 모델을 만들고 난 다음에 살펴볼 것입니다.  
다른 feature 로 split 하여 한번 살펴보겠습니다.  

In [28]:
f, ax = plt.subplots(2,2,figsize = (20,15))
sns.countplot('Embarked', data = df_train, ax = ax[0,0])
ax[0,0].set_title('(1) No. Of Passengers Boarded')
sns.countplot('Embarked', hue = 'Sex', data = df_train, ax = ax[0,1])
ax[0,1].set_title('(2) Male-Female Split for Embarked')
sns.countplot('Embarked', hue = 'Survived', data = df_train, ax = ax[1,0])
ax[1,0].set_title('(3) Embarked vs Survived')
sns.countplot('Embarked', hue = 'Pclass', data = df_train, ax = ax[1,1])
ax[1,1].set_title('(4) Embarked vs Pclass')
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

- Figure(1) - 전체적으로 봤을 때, S 에서 가장 많은 사람이 탑승했습니다.  
- Figure(2) - C와 Q 는 남녀의 비율이 비슷하고, S는 남자가 더 많습니다.  
- Figure(3) - 생존확률이 S 경우 많이 낮은 걸 볼 수 있습니다. (이전 그래프에서 봤었습니다)  
- Figure(4) - Class 로 split 해서 보니, C가 생존확률이 높은건 클래스가 높은 사람이 많이 타서 그렇습니다. S는 3rd class 가 많아서 생존확률이 낮게 나옵니다.  

## 2.7 Family - SibSp(형제 자매) + Parch(부모, 자녀)
SibSp와 Parch를 합하면 Family 가 될 것입니다. Family 로 합쳐서 분석해봅시다

In [29]:
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] + 1
# 자신을 포함해야하니 1을 더한다.
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] + 1

In [30]:
print("Maximum size of Family: ", df_train['FamilySize'].max())
print("Minimum size of Family: ", df_train['FamilySize'].min())

FamilySize 와 생존의 관계를 한번 살펴봅시다

In [31]:
f, ax = plt.subplots(1,3,figsize=(40,10))
sns.countplot('FamilySize', data = df_train, ax = ax[0])
ax[0].set_title('(1) No. Of Passengers Boarded', y = 1.02)
sns.countplot('FamilySize', hue = 'Survived', data = df_train, ax = ax[1])
ax[1].set_title('(2) Survived countplot dep ending on FamilySize', y=1.02)
df_train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index = True).mean().sort_values(by = 'Survived', ascending = False).plot.bar(ax = ax[2])
ax[2].set_title('(3) Survived rate depending on FamilySize', y = 1.02)

plt.subplots_adjust(wspace = 0.2, hspace = 0.5)
plt.show()

- Figure (1) - 가족크기가 1-11까지 있음을 볼 수 있습니다. 대부분 1명이고 그 다음으로 2, 3, 4명입니다.  
- Figure (2), (3) - 가족 크기에 따른 생존비교입니다. 가족이 4명인 경우가 가장 생존확률이 높습니다. 가족수가 많아질수록, (5, 6, 7, 8, 11) 생존확률이 낮아지네요. 가족수가 너무 작아도(1), 너무 커도(5, 6, 8, 11) 생존 확률이 작네요. 3-4명 선에서 생존확률이 높은 걸 확인할 수 있습니다.

## 2.8 Fare
Fare 는 탑승요금이며, contious feature 입니다. 한번 histogram 을 그려보겠습니다.

In [32]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
g = sns.distplot(df_train['Fare'], color = 'b', label = 'Skewness : {:.2f}'.format(df_train['Fare'].skew()), ax = ax)
g = g.legend(loc = 'best')

그려진 그래프를 보면, distribution이 매우 비대칭이다.(high skewness)  
만약 이대로 모델에 넣어준다면 자칫 모델이 잘못 학습할 수도 있다.  
몇개 없는 outlier 에 대해서 너무 민감하게 반응한다면, 실제 예측 시에 좋지 못한 결과를 부를 수 있다.  

outlier의 영향을 줄이기 위해 Fare 에 log 를 취하겠습니다.  
여기서 우리는 pandas 의 유용한 기능을 사용할 겁니다. dataFrame 의 특정 columns 에 공통된 작업(함수)를 적용하고 싶으면 아래의 map, 또는 apply 를 사용하면 매우 손쉽게 적용할 수 있습니다.  
우리가 지금 원하는 것은 Fare columns 의 데이터 모두를 log 값 취하는 것인데, 파이썬의 간단한 lambda 함수를 이용해 간단한 로그를 적용하는 함수를 map 에 인수로 넣어주면, Fare columns 데이터에 그대로 적용이 됩니다. (매우 유용한 기능이니 꼭 숙지!)  

In [33]:
df_test.loc[df_test.Fare.isnull(), 'Fare'] = df_test['Fare'].mean()
# testset에 있는 nan value를 평균값으로 치환한다.

df_train['Fare'] = df_train['Fare'].map(lambda i: np.log(i) if i > 0 else 0)
df_test['Fare'] = df_test['Fare'].map(lambda i: np.log(i) if i > 0 else 0)

In [34]:
fig, ax = plt.subplots(1,1,figsize = (8,8))
g = sns.distplot(df_train['Fare'], color = 'b', label = 'Skewness : {:.2f}'.format(df_train['Fare'].skew()), ax = ax )
g = g.legend(loc = 'best')

log 를 취하니, 이제 비대칭성이 많이 사라진 것을 볼 수 있습니다.  
우리는 이런 작업을 사용해 모델이 좀 더 좋은 성능을 내도록 할 수 있습니다.  
사실 방금한 것은 feature engineering 에 들어가는 부분인데, 여기서 작업했습니다.  
모델을 학습시키기 위해, 그리고 그 모델의 성능을 높이기 위해 feature 들에 여러 조작을 가하거나, 새로운 feature를 추가하는 것을 feature engineering 이라고 하는데, 우리는 이제 그것을 살펴볼 것입니다.

## 2.9 Cabin
이 feature 는 NaN 이 대략 80% 이므로, 생존에 영향을 미칠 중요한 정보를 얻어내기가 쉽지는 않습니다.  
그러므로 우리가 세우려는 모델에 포함시키지 않도록 하겠습니다.

In [35]:
df_train.head()

## 2.10 Ticket
이 feature 는 NaN 은 없습니다.  
일단 string data 이므로 우리가 어떤 작업들을 해주어야 실제 모델에 사용할 수 있는데, 이를 위해선 사실 아이디어가 필요합니다.

In [36]:
df_train['Ticket'].value_counts()